In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from datasets import load_dataset

In [ ]:
train_data = load_dataset("wmt16", "de-en", split="train[:50000]")
val_data = load_dataset("wmt16", "de-en", split="validation")
test_data = load_dataset("wmt16", "de-en", split="test")

de_tokenizer = get_tokenizer("spacy", language="de_core_news_sm")
en_tokenizer = get_tokenizer("spacy", language="en_core_web_sm")

In [ ]:
def yield_tokens(data, tokenizer):
    for sample in data:
        yield tokenizer(sample['translation']['de'])
        yield tokenizer(sample['translation']['en'])


de_vocab = build_vocab_from_iterator(yield_tokens(train_data, de_tokenizer), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
en_vocab = build_vocab_from_iterator(yield_tokens(train_data, en_tokenizer), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
de_vocab.set_default_index(de_vocab["<unk>"])
en_vocab.set_default_index(en_vocab["<unk>"])